In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from llama_index.core.evaluation import SemanticSimilarityEvaluator

evaluator = SemanticSimilarityEvaluator(embed_model=embed_model)

In [30]:
index_name = f"512-countries"
index_response_path = f'csv/countries/{index_name}-index-response.csv'
gpt_response_path = f'csv/countries/gpt-4-turbo-response.csv'
question_path = f'csv/countries-questions.csv'

output_path = f'csv/countries/CSGA/CSGA-{index_name}-LLAMA3_1.csv'

In [34]:
import pandas as pd
test_df = pd.read_csv(index_response_path)
ga_df = pd.read_csv(gpt_response_path)
ga_answers = ga_df['gpt-4-turbo']

In [33]:
import csv

with open(index_response_path, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    csv_data = list(reader)
    titles = csv_data[0]
    questions = csv_data[1:]
    
    with open(question_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        csv_data = list(reader)

    # test ans
    # loop through the titles only odd onese)
    for i in range(1, len(titles), 2):
        print(titles[i])
        test_answers = test_df[f'{titles[i]}']

        test_answers = list(test_answers)
        ga_answers = list(ga_answers)

        csv_data[0].append(f"Test Answer : {titles[i]}")
        csv_data[0].append(f"GA Answer")
        csv_data[0].append(titles[i])

        # loop through the test answers 
        for index in range(0,len(test_answers)):
            try:
                print(f"Index :" + str(index))
                print(f"Test Answer: {test_answers[index]}")
                print(f"GA Answer: {ga_answers[index]}")

                result = evaluator.evaluate(response=test_answers[index],reference=ga_answers[index])

                with open(output_path, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    csv_data[index +1].append(test_answers[index])
                    csv_data[index +1].append(ga_answers[index])
                    csv_data[index +1].append(result.score)
                    writer.writerows(csv_data)
            except Exception as e:
                print(f"Error: {e}")
                with open(output_path, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    csv_data[index +1].append("Error")
                    csv_data[index +1].append("Error")
                    csv_data[index +1].append("Error")
                    writer.writerows(csv_data)

llama-3.1-8b-instruct top_k=3
Index :0
Test Answer: According to the context, some reasons cited for Afghanistan being among the least developed countries in the world include:

1. Its rough physical geography
2. Its landlocked status
3. Contemporary conflict
4. Political instability
GA Answer: Some reasons cited for Afghanistan being among the least developed countries in the world include:

1. Rough physical geography
2. Landlocked status
3. Contemporary conflict
4. Political instability
Index :1
Test Answer: I don't have information about Djamel Laouisset's book "A Retrospective Study of the Algerian Iron and Steel Industry".
GA Answer: Djamel Laouisset's book "A Retrospective Study of the Algerian Iron and Steel Industry" was published in New York City by Nova Publishers.
Index :2
Test Answer: According to the document, the following countries are part of the Middle East region:

1. Bahrain
2. Cyprus
3. Egypt
4. Iran
5. Iraq
6. Israeli Kurdistan
7. Israel
8. Jordan
9. Kuwait
10. Le

KeyboardInterrupt: 

In [29]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(output_path)

# Drop the first column and first row as they contain questions and labels
df_numeric = df.iloc[1:, 1:].astype(float)

# Calculate the mean of each column
column_means = df_numeric.mean()

# Find the column with the highest mean
max_mean_column = column_means.idxmax()
max_mean_value = column_means.max()

print("Column Means:", column_means)
print("\nThe column with the highest mean is:", max_mean_column)
print("Highest mean value:", max_mean_value)

# Create the csv file and also 
# Highest Top K label | Tok Value
# llama-3.1-8b-instruct top_k=22 | 0.9940877532663676

# Open the existing CSV file to read the data
with open('csv/countries/CSGA/analysis.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    csv_data = list(reader)  # Read all the existing data

# Append the new row
csv_data.append([f"{index_name}", f"{max_mean_column}", f"{max_mean_value}"])

# Open the CSV file again to write the updated data
with open('csv/countries/CSGA/analysis.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

Column Means: llama-3.1-8b-instruct top_k=3     0.868404
llama-3.1-8b-instruct top_k=4     0.862200
llama-3.1-8b-instruct top_k=5     0.866092
llama-3.1-8b-instruct top_k=6     0.874792
llama-3.1-8b-instruct top_k=7     0.867838
llama-3.1-8b-instruct top_k=8     0.872442
llama-3.1-8b-instruct top_k=9     0.868515
llama-3.1-8b-instruct top_k=10    0.872724
llama-3.1-8b-instruct top_k=11    0.875283
llama-3.1-8b-instruct top_k=12    0.867190
llama-3.1-8b-instruct top_k=13    0.858651
llama-3.1-8b-instruct top_k=14    0.875762
llama-3.1-8b-instruct top_k=15    0.856675
llama-3.1-8b-instruct top_k=16    0.870035
llama-3.1-8b-instruct top_k=17    0.870343
llama-3.1-8b-instruct top_k=18    0.866791
llama-3.1-8b-instruct top_k=19    0.861235
llama-3.1-8b-instruct top_k=20    0.854649
llama-3.1-8b-instruct top_k=21    0.861807
llama-3.1-8b-instruct top_k=22    0.869789
llama-3.1-8b-instruct top_k=23    0.868961
llama-3.1-8b-instruct top_k=24    0.869551
llama-3.1-8b-instruct top_k=25    0.8690